## 고도차이와 위치 데이터를 기반으로 경사'각' 계산

In [1]:
import pandas as pd

In [2]:
data_url = './make_file/자동차_도로_고도차이_포함_데이터.csv'

In [3]:
data = pd.read_csv(data_url)
data.head(1)

,도로명,도로 종류,시작점_위도,시작점_경도,종료점_위도,종료점_경도,중앙점_위도,중앙점_경도,터널 여부,교량 여부,서비스 도로 여부,도로 ID,속도 제한,차선 수,일방통행 여부,교차로 여부,시작점_고도,종료점_고도,고도의_차이
0,"['광장앞지하차도', '광장로']",3차로,37.545509,127.097772,37.546747,127.102031,37.546128,127.099902,yes,NaN,NaN,"[175320385, 175320419, 175320429]",NaN,NaN,NaN,NaN,39.830399,21.125259,18.705139


In [4]:
data.columns

Index(['도로명', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '터널 여부', '교량 여부', '서비스 도로 여부', '도로 ID', '속도 제한', '차선 수',
       '일방통행 여부', '교차로 여부', '시작점_고도', '종료점_고도', '고도의_차이'],
      dtype='object')

In [5]:
# 예제 데이터 (배열)
start_lat = data['시작점_위도'].to_list()
start_lon = data['시작점_경도'].to_list()
end_lat = data['종료점_위도'].to_list()
end_lon = data['종료점_경도'].to_list()
different_alt = data['고도의_차이'].to_list()

In [6]:
print(f'start_lat 길이 : {len(start_lat)}')
print(f'start_lon 길이 : {len(start_lon)}')
print(f'end_lat 길이 : {len(end_lat)}')
print(f'end_lon 길이 : {len(end_lon)}')
print(f'different_alt 길이 : {len(different_alt)}')

start_lat 길이 : 7216
start_lon 길이 : 7216
end_lat 길이 : 7216
end_lon 길이 : 7216
different_alt 길이 : 7216


In [7]:
import math

# 지구 반지름 (미터)
R = 6371000   

In [8]:
# 결과 저장 리스트
road_slope = []

In [9]:
for i in range(len(start_lat)):
    # 위경도를 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(math.radians, [start_lat[i], start_lon[i], end_lat[i], end_lon[i]])

    # 위도, 경도의 차이
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine 공식으로 수평 거리(d) 계산
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c  # 두 지점 간 수평 거리 (m)

    # 고도 차이 (h)
    h = different_alt[i]  

    # 경사각(θ) 계산 (라디안 → 도 변환)
    theta = math.degrees(math.atan2(h, d))

    # 결과 저장
    road_slope.append(theta)

    if(i < 5) :
        # 디버깅 출력
        print(f"Point {i + 1}: d = {d:.4f} m, h = {h:.4f} m, θ = {theta:.4f}°")


Point 1: d = 399.8650 m, h = 18.7051 m, θ = 2.6783°
Point 2: d = 117.1956 m, h = 10.7009 m, θ = 5.2171°
Point 3: d = 177.0293 m, h = 8.9262 m, θ = 2.8865°
Point 4: d = 10.9304 m, h = 0.0540 m, θ = 0.2828°
Point 5: d = 26.9216 m, h = 0.0700 m, θ = 0.1489°


In [10]:
print(f'변환한 road_slope의 길이 : {len(road_slope)}')

변환한 road_slope의 길이 : 7216


In [11]:
new_add_df = pd.DataFrame({
    '경사각' : road_slope
})

In [12]:
data['경사각'] = new_add_df

In [13]:
data.columns

Index(['도로명', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '터널 여부', '교량 여부', '서비스 도로 여부', '도로 ID', '속도 제한', '차선 수',
       '일방통행 여부', '교차로 여부', '시작점_고도', '종료점_고도', '고도의_차이', '경사각'],
      dtype='object')

In [14]:
new_order = [
    '도로명', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도',
    '중앙점_위도', '중앙점_경도', '시작점_고도', '종료점_고도', '고도의_차이', '경사각'
] + [col for col in data.columns if col not in [
    '도로명', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도',
    '중앙점_위도', '중앙점_경도', '시작점_고도', '종료점_고도', '고도의_차이', '경사각'
]]

# 컬럼 순서 재배열
data = data[new_order]


In [15]:
data.columns

Index(['도로명', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '시작점_고도', '종료점_고도', '고도의_차이', '경사각', '터널 여부', '교량 여부',
       '서비스 도로 여부', '도로 ID', '속도 제한', '차선 수', '일방통행 여부', '교차로 여부'],
      dtype='object')

In [16]:
data.head(5)

,도로명,도로 종류,시작점_위도,시작점_경도,종료점_위도,종료점_경도,중앙점_위도,중앙점_경도,시작점_고도,종료점_고도,고도의_차이,경사각,터널 여부,교량 여부,서비스 도로 여부,도로 ID,속도 제한,차선 수,일방통행 여부,교차로 여부
0,"['광장앞지하차도', '광장로']",3차로,37.545509,127.097772,37.546747,127.102031,37.546128,127.099902,39.830399,21.125259,18.705139,2.678266,yes,NaN,NaN,"[175320385, 175320419, 175320429]",NaN,NaN,NaN,NaN
1,워커힐로,주거지 도로,37.545509,127.097772,37.546539,127.098053,37.546024,127.097913,39.830399,50.531334,10.700935,5.217114,NaN,NaN,NaN,516647748,NaN,NaN,NaN,NaN
2,워커힐로,주거지 도로,37.545509,127.097772,37.543981,127.097208,37.544745,127.097490,39.830399,30.904158,8.926241,2.886546,NaN,NaN,NaN,516647752,NaN,NaN,NaN,NaN
3,아차산로,2차로,37.536546,127.083544,37.536555,127.083667,37.536550,127.083605,16.220312,16.274267,0.053955,0.282824,NaN,NaN,NaN,1199529109,50.0,3,True,NaN
4,자양로,주도로,37.536546,127.083544,37.536310,127.083613,37.536428,127.083579,16.220312,16.150337,0.069975,0.148924,NaN,NaN,NaN,1193259435,NaN,3,True,NaN


In [17]:
data.to_csv('./make_file/자동차_도로_고도차이(경사각)_포함_데이터.csv', encoding='euc-kr')